### Playback raw motions for `motion-inbetweening`

In [1]:
import time,pickle,mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,r2quat,MultiSliderClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.6]


### Parse `scene_common_rig_v2.xml`

In [2]:
xml_path = '../asset/common_rig/scene_common_rig_v2.xml'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=False)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs: env.model.geom(geom_idx).rgba = [0.9,0.9,0.9,0.5]
print ("Done.")
# Set which joints to control
rev_joint_names = env.ctrl_names 
joint_idxs_fwd = env.get_idxs_fwd(joint_names=rev_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=rev_joint_names)
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

Done.


In [3]:
pkl_path = '../data/VAAI_DIRECT_09_02_a_M1.pkl'
# pkl_path = '../data/VAAI_DIRECT_03_01_a_M1.pkl'

# pkl_path = '../data/VAAI_DIRECT_10_01_c_M1.pkl'
# pkl_path = '../data/VAAI_DIRECT_20_01_a_M1.pkl'
with open(pkl_path, 'rb') as f:
    data = pickle.load(f)
qs = data['qpos'][:,list(range(0,4))+list(range(5,12))+list(range(13,37))] # [L x 35]
quat_roots = r2quat(data['root_rot'])
p_roots = data['root_translation']
L = qs.shape[0]

In [4]:
tick_slider = MultiSliderClass(
    n_slider      = 1,
    title         = 'Tick slider',
    window_width  = 500,
    window_height = 100,
    x_offset      = 100,
    y_offset      = 0,
    slider_width  = 350,
    label_texts   = ['Tick'],
    slider_mins   = [0],
    slider_maxs   = [L-1],
    slider_vals   = [0],
    resolution    = 0.5,
    ADD_PLAYBACK  = True,
    VERBOSE       = False
)

In [5]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
while env.is_viewer_alive():
    # Update 
    time.sleep(1e-10) # little delay helps
    tick_slider.update()
    if tick_slider.PLAYBACK: # play mode
        tick = min(tick+1,L-1)
        if tick == (L-1): tick_slider.reset_playback()
        tick_slider.set_slider_values([tick])
    else: # stop mode
        slider_val = tick_slider.get_slider_values()
        tick = int(slider_val[0])
    
    # FK
    q = qs[tick,:] # [35]
    p_root = p_roots[tick,:] # [3]
    quat_root = quat_roots[tick,:] # [4] quaternion
    env.set_p_root(root_name='base',p=p_root)
    env.set_quat_root(root_name='base',quat=quat_root)
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    if env.loop_every(tick_every=100) or tick_slider.PLAYBACK:
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
# Close MuJoCo viewer
env.close_viewer()
tick_slider.close()
print ("Done.")

Pressed ESC
Quitting.
Done.
